In [ ]:
HOME_DIR = "/content/drive/MyDrive/PhD research/LLM Privacy Policy"

In [ ]:
!pip -q install llama-index llama-index-embeddings-huggingface llama-index-llms-llama-cpp pypdf
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -q install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import time
from glob import glob
import pandas as pd

from transformers import LlamaForCausalLM, LlamaTokenizer
from llama_index.core import Prompt, StorageContext, load_index_from_storage, Settings, VectorStoreIndex, SimpleDirectoryReader, set_global_tokenizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP

from transformers import AutoTokenizer
from IPython.display import Markdown, display

# LLM Model

In [ ]:
text_embedding_model = 'thenlper/gte-base'  #Alt: thenlper/gte-base, jinaai/jina-embeddings-v2-base-en
llm_url = 'https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'

# Load models and service context
embed_model = HuggingFaceEmbedding(model_name=text_embedding_model)
llm = LlamaCPP(model_url=llm_url, temperature=0.7, max_new_tokens=256, context_window=4096, generate_kwargs = {"stop": ["<s>", "[INST]", "[/INST]"]}, model_kwargs={"n_gpu_layers": -1}, verbose=True)
# service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model, chunk_size=512)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

total size (MB): 4081.0


3892it [00:30, 125.62it/s]                          
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /tmp/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loa

In [ ]:
def extractQueryAnsFromPdf(query, pdf_path):
  filename_fn = lambda filename: {'file_name': os.path.basename(pdf_path)}
  loader = SimpleDirectoryReader(input_files=[pdf_path], file_metadata=filename_fn)
  documents = loader.load_data()

  # Indexing
  start_time = time.time()

  index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, llm=llm)

  end_time = time.time()
  elapsed_time = end_time - start_time
  print(f"Elapsed indexing time: {elapsed_time:.2f} s")

  query_str = "What are the data the company collect?"
  query_engine = index.as_query_engine(similarity_top_k=1, llm=llm)
  # use this for testing
  vector_retriever = index.as_retriever(similarity_top_k=1)
  response = query_engine.query(query_str)
  return str(response)

In [ ]:
privacyPolicyPaths = glob("/content/drive/MyDrive/PhD research/LLM Privacy Policy/Codes/automated privacy polixy/*.pdf")
len(privacyPolicyPaths)

27

In [ ]:
dictAll = []
for pdf_path in privacyPolicyPaths[20:]:
  query = "What are the data the company collect?"
  response = extractQueryAnsFromPdf(query, pdf_path)
  print(response)

  dictAll.append({'file_name': os.path.basename(pdf_path), 'response': response})

Elapsed indexing time: 29.67 s


llama_perf_context_print:        load time =  193311.15 ms
llama_perf_context_print: prompt eval time =  193304.77 ms /   683 tokens (  283.02 ms per token,     3.53 tokens per second)
llama_perf_context_print:        eval time =  154901.50 ms /   255 runs   (  607.46 ms per token,     1.65 tokens per second)
llama_perf_context_print:       total time =  348394.10 ms /   938 tokens


 Based on the context information provided, Wipro collects the following data:
1. Personal data processing activities: Wipro collects information about the personal data processing activities of its clients, including the type of data processed, the purpose of processing, and the duration of processing.
2. Privacy expectations: Wipro collects information about the privacy expectations of its clients, including their preferences regarding data privacy and security.
3. Contractual obligations: Wipro collects information about the contractual obligations of its clients, including the terms and conditions of their agreements and the regulatory requirements that apply to their data processing activities.
4. Vendor risk assessments: Wipro collects information about the risk assessments conducted on its vendors, including the results of these assessments and the measures taken to mitigate any identified risks.
5. Data transmission and storage: Wipro collects information about the methods used

Llama.generate: 15 prefix-match hit, remaining 674 prompt tokens to eval
llama_perf_context_print:        load time =  193311.15 ms
llama_perf_context_print: prompt eval time =  187828.45 ms /   674 tokens (  278.68 ms per token,     3.59 tokens per second)
llama_perf_context_print:        eval time =   30167.68 ms /    51 runs   (  591.52 ms per token,     1.69 tokens per second)
llama_perf_context_print:       total time =  218022.92 ms /   725 tokens


 Based on the provided context information, the company collects Personal Data, including sensitive or special categories of Personal Data, as defined in applicable Data Privacy Laws. They may collect Personal Data necessary for their contractual obligations and to perform their services.
Elapsed indexing time: 29.24 s


...

Llama.generate: 15 prefix-match hit, remaining 582 prompt tokens to eval
llama_perf_context_print:        load time =  193311.15 ms
llama_perf_context_print: prompt eval time =  159682.36 ms /   582 tokens (  274.37 ms per token,     3.64 tokens per second)
llama_perf_context_print:        eval time =  154872.82 ms /   252 runs   (  614.57 ms per token,     1.63 tokens per second)
llama_perf_context_print:       total time =  314737.34 ms /   834 tokens



The company collects the following data:
1. Personal Data from Client Account Set-Up and Account Use: The company collects certain Personal Data that clients choose to provide about their Personnel, including a user ID, name, email address, phone number, address, information about a Personnel’s skills and professional experience, and other Personal Data.
2. Location Data: The company collects data related to the geographic location of the mobile device on which the Cloud Service is being used by Personnel (“Location Data”). The company uses various technologies to determine location, including GPS signals, Wi-Fi access points, and cell tower IDs. If you choose to use these Location Based Services, you consent to the company’s use of Location Data. If you don’t want to receive your Location Data, you should be able to turn off location-sharing features on your mobile device.
3. Personnel Mobile Device Data: The company may collect other limited information from the mobile device of Per

Llama.generate: 18 prefix-match hit, remaining 478 prompt tokens to eval
llama_perf_context_print:        load time =  193311.15 ms
llama_perf_context_print: prompt eval time =  134737.68 ms /   478 tokens (  281.88 ms per token,     3.55 tokens per second)
llama_perf_context_print:        eval time =  105812.13 ms /   169 runs   (  626.11 ms per token,     1.60 tokens per second)
llama_perf_context_print:       total time =  240656.25 ms /   647 tokens


 The company collects the following data:
* Personal information (i.e. information that could be used to contact You directly (without using the Lyft Platform) such as full name, postal address, phone number, credit/debit card information, or email address)
* Demographic information (i.e. information that You submit, or that We collect, that is not personal information; this may include, but is not limited to, zip code, hometown, gender, username, age/birth date, browsing history information, searching history information, and registration history information.
* Contact information of Your friends (if You choose to connect Your contacts and address book information with Lyft)
* Login credentials to Your social network accounts (if You choose to connect those accounts with your Lyft account).


Failed to load file /content/drive/MyDrive/PhD research/LLM Privacy Policy/Codes/automated privacy polixy/FedEx_Privacy_Notice.pdf with error: RetryError[<Future at 0x7b9230fc43d0 state=finished raised PdfStreamError>]. Skipping...
Elapsed indexing time: 0.00 s
Empty Response


In [ ]:
dictAll

[{'file_name': 'data-privacy-at-wipro.pdf',
  'response': ' Based on the context information provided, Wipro collects the following data:\n1. Personal data processing activities: Wipro collects information about the personal data processing activities of its clients, including the type of data processed, the purpose of processing, and the duration of processing.\n2. Privacy expectations: Wipro collects information about the privacy expectations of its clients, including their preferences regarding data privacy and security.\n3. Contractual obligations: Wipro collects information about the contractual obligations of its clients, including the terms and conditions of their agreements and the regulatory requirements that apply to their data processing activities.\n4. Vendor risk assessments: Wipro collects information about the risk assessments conducted on its vendors, including the results of these assessments and the measures taken to mitigate any identified risks.\n5. Data transmissio

In [ ]:
# Convert dictionary to DataFrame
df = pd.DataFrame(dictAll)

# Save to CSV
df.to_excel("/content/drive/MyDrive/PhD research/LLM Privacy Policy/Codes/last7.xlsx", index=False)

In [ ]:
df

,file_name,response
0,data-privacy-at-wipro.pdf,"Based on the context information provided, Wi..."
1,Data-Privacy-Supplement.pdf,"Based on the provided context information, th..."
2,SAP_Discovery_Center_Privacy_Statement.pdf,"Based on the context information provided, th..."
3,bcr-privacy-code-051719.pdf,\nThe company collects the following data:\n\n...
4,cloud-services-privacy-policy.pdf,\nThe company collects the following data:\n1....
5,Lyft_privacy_policy_dated_9.10.12.pdf,The company collects the following data:\n* P...
6,FedEx_Privacy_Notice.pdf,Empty Response
